## Тестовое задание. Бурцев Марк.

In [1]:
import pandas as pd

In [2]:
path = ''
dataset = pd.read_csv(path+'lap_times.csv')

In [3]:
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490904 entries, 0 to 490903
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        490904 non-null  int64 
 1   driverId      490904 non-null  int64 
 2   lap           490904 non-null  int64 
 3   position      490904 non-null  int64 
 4   time          490904 non-null  object
 5   milliseconds  490904 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 22.5+ MB


,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


По условиям задачи необходимо выбрать для исследования одну гонку. Выберем гонку под номером 841.

In [4]:
dataset = dataset.query('raceId == 841')

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1083 entries, 0 to 1082
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   raceId        1083 non-null   int64 
 1   driverId      1083 non-null   int64 
 2   lap           1083 non-null   int64 
 3   position      1083 non-null   int64 
 4   time          1083 non-null   object
 5   milliseconds  1083 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 59.2+ KB


## Суммарное время заезда для каждого гонщика

Создадим новый стобец, который будет содержать точное время. К столбцу `time` добавим данные из столбца `milliseconds`. После преобразуем получившуюся строку в формат, удобный для работы с временем.

In [6]:
dataset['exact_time'] = dataset['time'] + dataset['milliseconds'].astype('str')
dataset['exact_time'] = pd.to_datetime(dataset['exact_time'], format = '%M:%S.%f') - pd.to_datetime('1900-01-01', format = '%Y-%m-%d')

In [7]:
dataset.head()

,raceId,driverId,lap,position,time,milliseconds,exact_time
0,841,20,1,1,1:38.109,98109,00:01:38.109981
1,841,20,2,1,1:33.006,93006,00:01:33.006930
2,841,20,3,1,1:32.713,92713,00:01:32.713927
3,841,20,4,1,1:32.803,92803,00:01:32.803928
4,841,20,5,1,1:32.342,92342,00:01:32.342923


Создадим таблицу, в которой для каждого номера участница будет суммарное время его заезда.

In [8]:
sum_time = dataset.pivot_table(index = 'driverId',
                            values = 'exact_time', 
                            aggfunc= 'sum')

In [9]:
sum_time

,exact_time
driverId,
1,01:29:52.606992
2,01:30:43.952367
3,00:35:21.894315
4,01:30:02.081036
5,00:31:16.474750
10,01:31:02.675756
13,01:30:55.494647
15,01:30:53.585457
16,01:29:54.843066


## Топ-3 гонщика по каждому кругу

Найдем топ-3 гонщика для каждого круга с помощью сводной таблицы, в которой строки - номер круга, столбцы - позиция в зачете, а на пересечении - номер гонщика.

Из получившейся таблицы выделим первые три места.

In [10]:
top = dataset.pivot_table(index = 'lap', columns = 'position', values = 'driverId', aggfunc = 'first')
top = top.iloc[:, [0,1,2]].astype('int')

In [11]:
top

position,1,2,3
lap,,,
1,20,1,17
2,20,1,17
3,20,1,17
4,20,1,17
5,20,1,17
6,20,1,17
7,20,1,17
8,20,1,17
9,20,1,17


## Для каждого гонщика – круг, в котором у него был самый большой положительный рывок

Для начала создадим сводную таблицу, где индексами будут номера участников, столбцами - номера кругов, а в значениях таблицы будет позиция участника на данном круге.

In [12]:
max_delta_lap = dataset.pivot_table(index = 'driverId', columns = 'lap', values = 'position', aggfunc = 'first')

Для удобства дальнейшей работы переименуем стобцы.

In [13]:
max_delta_lap.columns = range(1,59)

Теперь вычислим для каждого участника требуемую разницу на каждом круге в сравнении с предыдущим. Для этого сдвинем значения в таблице на одну колонку вправо и вычтем из полученной таблицы начальную. Таким образом в результате мы будем иметь таблицу, где для каждого участника на каждом круге будет указана та разница в зачете мест, которую он приобрел за данный круг.

In [14]:
max_delta_lap = max_delta_lap.shift(1, axis = 1) - max_delta_lap.shift(0, axis = 1)

Создадим функцию, на вход которой будет подаваться строка датафрейма. Внутри функция будет находить максимальную положительную разницу в местах, и на выход подавать значение круга (или значения, если максимальная разница была найдена несколько раз), в котором была найдена максимальная разница.

In [15]:
def lap_max(row):
    max_delta = 0
    lap = []
    for i in range(2, 58):
        if row[i+1] > max_delta:
            max_delta = row[i+1]
    for i in (range(2,59)):
        if row[i] == max_delta:
            lap.append(i)
    return lap

Создадим с помощью функции в таблице столбец, в котором для каждого индекса будет указан номер круга, на котором была зафиксирована максимальная положительная разница в зачете. Если значение максимальной разницы будет одинаково на нескольких кругах, то в полученном столбце мы будем иметь список из номеров этих кругов.

Также удалим более ненужные колонки.

In [16]:
max_delta_lap['top_lap'] = max_delta_lap.apply(lap_max, axis = 1)
max_delta_lap = max_delta_lap.drop([i for i in range(1,59)], axis = 1)

In [17]:
max_delta_lap

,top_lap
driverId,
1,[14]
2,[23]
3,[12]
4,"[14, 16]"
5,"[10, 13, 14, 15, 16, 18]"
10,"[17, 20]"
13,[16]
15,"[10, 19, 20, 23, 49]"
16,[23]


## Динамикуа мест по кругам для гонщиков, завершивших заезд в десятке лучших

In [18]:
len(dataset[dataset['lap'] == 58])

9

Видно, что до последнего круга доехало только 9 участников. Чтобы просмотреть топ 10, сделаем выборку с условием того, что участник добрался до предпоследнего круга.

Создадим список номеров участников, которые входят в топ-10.

In [19]:
top_10 = dataset.query('(lap == 57) & (position < 11)')['driverId']

Выберем из общего датасета строки, в которых номер участника есть в списке десяти лучших.

In [20]:
dynamics = dataset.query('driverId in @top_10')

Создадим сводную таблицу, в которой строки представляют собой участников, столбцы - номер круга. На пересечении i-той строки и j-того столбца - место участника, номер которого расположен в i-том индексе на j-том круге.

In [21]:
dynamics = dynamics.pivot_table(index = 'driverId', columns = 'lap', values = 'position', aggfunc = 'first')
dynamics = dynamics.fillna(10).astype('int')

In [22]:
dynamics

lap,1,2,3,4,5,6,7,8,9,10,...,49,50,51,52,53,54,55,56,57,58
driverId,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
4,9,8,8,8,7,7,7,7,7,7,...,4,4,4,4,4,4,4,4,4,4
13,5,5,5,5,5,5,5,5,5,5,...,10,10,10,10,10,9,9,9,9,9
17,3,3,3,3,3,3,3,3,3,3,...,5,5,5,5,5,5,5,5,5,5
18,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
20,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
67,11,11,11,11,11,11,10,10,10,10,...,9,9,9,9,9,10,10,10,10,10
155,8,9,9,9,9,9,9,9,9,9,...,8,8,8,8,8,8,8,8,8,8
808,4,4,4,4,4,4,4,4,4,4,...,3,3,3,3,3,3,3,3,3,3


Таким образом каждая строка полученной таблицы дает информацию о том, какое место каждый участник занимална каждом круге.